In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pathlib

from ALLCools.plot import *
from wmb import *

In [ ]:
mc_annot = cemba.get_mc_annot()
m3c_annot = cemba.get_m3c_annot()

In [ ]:
mc_l4_to_inte_group = pd.read_csv('mc_l4_to_integration_group.csv', index_col=0).squeeze()
mc_l4_to_inte_l1 = mc_l4_to_inte_group.apply(lambda i: '_'.join(str(i).split('_')[:1]))
mc_l4_to_inte_l2 = mc_l4_to_inte_group.apply(lambda i: '_'.join(str(i).split('_')[:2]))
mc_l4_to_inte_l4 = mc_l4_to_inte_group.apply(lambda i: '_'.join(str(i).split('_')[:3]))

m3c_l4_to_inte_group = pd.read_csv('m3c_l4_to_integration_group.csv', index_col=0).squeeze()
m3c_l4_to_inte_l1 = m3c_l4_to_inte_group.apply(lambda i: '_'.join(str(i).split('_')[:1]))
m3c_l4_to_inte_l2 = m3c_l4_to_inte_group.apply(lambda i: '_'.join(str(i).split('_')[:2]))
m3c_l4_to_inte_l4 = m3c_l4_to_inte_group.apply(lambda i: '_'.join(str(i).split('_')[:3]))

## Read Confusion Matrix

- Load L1 to L4 confusion matrix
- Turn all the confusion matrix into mC and m3c L4 resolution

### L1 Confusion Matrix

In [ ]:
l1_confusion_matrix = pd.read_hdf(f'L1_confusion_matrx.L4_clusters.hdf') 

## L2 Confuion Matrix

In [ ]:
l2_confusion_matrix = pd.read_hdf('L2_confusion_matrx.L4_clusters.hdf')

## L4 Confusion Matrix
- read all L4 overlap score matrix
- replace value of l2_confusion_matrix with L4 values

In [ ]:
l4_confusion_matrix = pd.read_hdf('L4_confusion_matrx.L4_clusters.hdf')

## Plot Confusion matrix 

- Three confusion matrix showing the process from L1 to L4 integration

In [ ]:
palette = 'magma_r'

cmap = plt.get_cmap(palette).copy()
norm = mpl.colors.Normalize(vmin=0, vmax=1)

scalar_mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)

In [ ]:
fig, ax = plt.subplots(figsize=(0.3, 2), dpi=300)
fig.colorbar(scalar_mappable,
             cax=ax, label='Overlap Score')

### L1

In [ ]:
mat = l1_confusion_matrix.loc[m3c_l4_to_inte_l1.sort_values().index, mc_l4_to_inte_l1.sort_values().index]

In [ ]:
# tidy data
plot_data = mat.unstack().reset_index()
plot_data.columns = ['mC', 'm3c', 'OS']
plot_data = plot_data[plot_data['OS'] > 0].copy()

y_map = {c: i for i, c in enumerate(mat.index)}
plot_data['y'] = plot_data['m3c'].map(y_map)

x_map = {c: i for i, c in enumerate(mat.columns)}
plot_data['x'] = plot_data['mC'].map(x_map)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=300)
sns.scatterplot(data=plot_data.sample(100000),
                x='x',
                y='y',
                linewidth=0,
                size='OS',
                size_norm=(0, 1),
                sizes=(0, 1),
                hue='OS',
                hue_norm=(0, 1),
                palette=palette,
                cmap=cmap,
                legend=None)
ax.set(xticks=[],
       xlabel='mC Clusters',
       yticks=[],
       ylabel='m3c Clusters')

ax.set_title(f'L1 Integration Overlap Scores\n'
             f'{m3c_l4_to_inte_l1.unique().size} Integration Groups',
             fontsize=10)

pass

fig.savefig('L1_confusion_matrix.png')

### L2

In [ ]:
# add missing mC L4 cluster
columns_to_add = set(l1_confusion_matrix.columns) ^ set(l2_confusion_matrix.columns)
for i in columns_to_add:
    l2_confusion_matrix[i] = l1_confusion_matrix[i]

In [ ]:
# add m3C cluster
rows_to_add =  set(l1_confusion_matrix.index) ^ set(l2_confusion_matrix.index)
for i in rows_to_add:
    l2_confusion_matrix.loc[i] = l1_confusion_matrix.loc[i]

In [ ]:
mat = l2_confusion_matrix.loc[m3c_l4_to_inte_l2.sort_values().index, mc_l4_to_inte_l2.sort_values().index]

In [ ]:
# tidy data
plot_data = mat.unstack().reset_index()
plot_data.columns = ['mC', 'm3c', 'OS']
plot_data = plot_data[plot_data['OS'] > 0].copy()

y_map = {c: i for i, c in enumerate(mat.index)}
plot_data['y'] = plot_data['m3c'].map(y_map)

x_map = {c: i for i, c in enumerate(mat.columns)}
plot_data['x'] = plot_data['mC'].map(x_map)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=300)
sns.scatterplot(data=plot_data,
                x='x',
                y='y',
                linewidth=0,
                size='OS',
                size_norm=(0, 1),
                sizes=(0, 1),
                hue='OS',
                hue_norm=(0, 1),
                palette=palette,
                cmap=cmap,
                legend=None)
ax.set(xticks=[],
       xlabel='mC Clusters',
       yticks=[],
       ylabel='m3c Clusters')

ax.set_title(f'L2 Integration Overlap Scores\n'
             f'{m3c_l4_to_inte_l2.unique().size} Integration Groups',
             fontsize=10)

pass
fig.savefig('L2_confusion_matrix.png')

## L4

In [ ]:
# add missing mC L4 cluster
columns_to_add = set(l1_confusion_matrix.columns) ^ set(l4_confusion_matrix.columns)
for i in columns_to_add:
    l4_confusion_matrix[i] = l1_confusion_matrix[i]

In [ ]:
# add m3C cluster
rows_to_add =  set(l1_confusion_matrix.index) ^ set(l4_confusion_matrix.index)
for i in rows_to_add:
    l4_confusion_matrix.loc[i] = l1_confusion_matrix.loc[i]

In [ ]:
mat = l4_confusion_matrix.loc[m3c_l4_to_inte_l4.sort_values().index, mc_l4_to_inte_l4.sort_values().index]

In [ ]:
# tidy data
plot_data = mat.unstack().reset_index()
plot_data.columns = ['mC', 'm3c', 'OS']
plot_data = plot_data[plot_data['OS'] > 0].copy()

y_map = {c: i for i, c in enumerate(mat.index)}
plot_data['y'] = plot_data['m3c'].map(y_map)

x_map = {c: i for i, c in enumerate(mat.columns)}
plot_data['x'] = plot_data['mC'].map(x_map)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=300)
sns.scatterplot(data=plot_data, #plot_data.sample(100000)
                x='x',
                y='y',
                linewidth=0,
                size='OS',
                size_norm=(0, 1),
                sizes=(0, 1),
                hue='OS',
                hue_norm=(0, 1),
                palette=palette,
                cmap=cmap,
                legend=None)
ax.set(xticks=[],
       xlabel='mC Clusters',
       yticks=[],
       ylabel='m3c Clusters')

ax.set_title(f'L4 Integration Overlap Scores\n'
             f'{m3c_l4_to_inte_l4.unique().size} Integration Groups',
             fontsize=10)
pass

fig.savefig('L4_confusion_matrix.png')